In [ ]:
import pandas as pd
import os
import re

In [ ]:
data_dir = '/Users/jk1/Downloads/data_saving/exclude_nan_outcome_False'

In [ ]:
data_filenames = [f for f in os.listdir(data_dir) if f.endswith('.csv') and 'timebin' in f and 'reassembled_pupillometry' in f]

pupillometry_df = pd.DataFrame()
for data_filename in data_filenames:
    # find timebin size with regex identifying pattern : _xh_
    timebin_size = int(re.search(r'_(\d+)h_', data_filename).group(1))
    data_is_normalized = int(('normalized' in data_filename) or ('normalised' in data_filename))
    outcome = '_'.join(data_filename.split('_')[0:2])

    df = pd.read_csv(os.path.join(data_dir, data_filename))
    df['timebin_size'] = timebin_size
    df['normalized'] = data_is_normalized
    df['outcome'] = outcome
    pupillometry_df = pd.concat([pupillometry_df, df], axis=0)
    
pupillometry_df = pupillometry_df.reset_index(drop=True)

In [ ]:
pupillometry_metrics = ['NPI', 'CV']
inter_eye_metrics = ['mean', 'min', 'max', 'delta']
# combine to get all metrics
single_timepoint_metrics = [f'{metric}_inter_eye_{metric_type}' for metric in pupillometry_metrics for metric_type in
                            inter_eye_metrics]
over_time_metrics = ['max', 'min', 'median']
# combine to get all metrics
timebin_metrics = [f'{metric}_timebin_{metric_type}' for metric in single_timepoint_metrics for metric_type in
                   over_time_metrics]

In [ ]:
# drop rows where all timebin_metrics are nan
pupillometry_df = pupillometry_df.dropna(subset=timebin_metrics, how='all')

# for every outcome and timebin size, count number of positive and negative labels
pupillometry_df.groupby(['outcome', 'timebin_size', 'normalized', 'label']).size().unstack(fill_value=0).reset_index()